In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import os

# Define the path to your dataset folder
dataset_path = 'Data/'

# Set parameters for image processing
batch_size = 32
img_height = 224
img_width = 224

# Create ImageDataGenerator instance for training and validation
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)  # Scale pixel values to [0, 1]

# Load training data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',   # For multi-class classification
    subset='training'           # Specify that this is for training data
)

# Load validation data
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',   # For multi-class classification
    subset='validation'         # Specify that this is for validation data
)


Found 10828 images belonging to 11 classes.
Found 2702 images belonging to 11 classes.


Model 1: Simple CNN
Here's a simple CNN model without optimization techniques.

In [10]:
# Define Model 1: A simple CNN
def create_simple_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(train_data.num_classes, activation='softmax')  # Output layer for multi-class classification
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

simple_cnn = create_simple_cnn()

# Train the model
simple_cnn.fit(train_data, validation_data=val_data, epochs=10)

# Save the model
simple_cnn.save('saved_models/model1.h5')


Epoch 1/10


C:\Users\Hp\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


339/339 ━━━━━━━━━━━━━━━━━━━━ 473s 1s/step - accuracy: 0.4507 - loss: 2.3341 - val_accuracy: 0.7620 - val_loss: 0.7123
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 396s 1s/step - accuracy: 0.8369 - loss: 0.4899 - val_accuracy: 0.8316 - val_loss: 0.5171
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 426s 1s/step - accuracy: 0.9338 - loss: 0.2167 - val_accuracy: 0.8272 - val_loss: 0.5639
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 458s 1s/step - accuracy: 0.9698 - loss: 0.1016 - val_accuracy: 0.8242 - val_loss: 0.6118
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 395s 1s/step - accuracy: 0.9916 - loss: 0.0379 - val_accuracy: 0.8453 - val_loss: 0.6059
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.9915 - loss: 0.0320 - val_accuracy: 0.8327 - val_loss: 0.6471
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.9833 - loss: 0.0524 - val_accuracy: 0.8368 - val_loss: 0.6997
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 355s 1s/step - accuracy: 0.9842 - loss: 0.0446 - val_accuracy: 0.832

Model 2: Optimized CNN
Here's the optimized CNN model with techniques like dropout, batch normalization, and data augmentation.

In [1]:
# Define Model 2: Optimized CNN with dropout and batch normalization
def create_optimized_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout layer for regularization
        Dense(train_data.num_classes, activation='softmax')  # Output layer for multi-class classification
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

optimized_cnn = create_optimized_cnn()

# Train the optimized model
optimized_cnn.fit(train_data, validation_data=val_data, epochs=10)

# Save the optimized model
optimized_cnn.save('saved_models/model2.h5')


NameError: name 'Sequential' is not defined

Step 3: Make Predictions


In [ ]:
# Load test data
test_data = datagen.flow_from_directory(
    'Data/test/',  # Adjust the path as necessary
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Ensure the order is maintained for predictions
)

# Evaluate both models on the test data
simple_eval = simple_cnn.evaluate(test_data)
optimized_eval = optimized_cnn.evaluate(test_data)

print(f"Simple CNN Test Accuracy: {simple_eval[1] * 100:.2f}%")
print(f"Optimized CNN Test Accuracy: {optimized_eval[1] * 100:.2f}%")
